# On-Time Flight Performance with GraphFrames for Apache Spark
This notebook provides an analysis of On-Time Flight Performance and Departure Delays data using GraphFrames for Apache Spark.

Source Data: 
* [OpenFlights: Airport, airline and route data](http://openflights.org/data.html)
* [United States Department of Transportation: Bureau of Transportation Statistics (TranStats)](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time)
 * Note, the data used here was extracted from the US DOT:BTS between 1/1/2014 and 3/31/2014*

References:
* [GraphFrames User Guide](http://graphframes.github.io/user-guide.html)
* [GraphFrames: DataFrame-based Graphs (GitHub)](https://github.com/graphframes/graphframes)
* [D3 Airports Example](http://mbostock.github.io/d3/talk/20111116/airports.html)

In [1]:
%%configure -f
{ "conf": {"spark.jars.packages": "graphframes:graphframes:0.3.0-spark2.0-s_2.11" }}

In [2]:
import os
sc.addPyFile(os.path.expanduser('./graphframes_graphframes-0.3.0-spark2.0-s_2.11.jar'))
from graphframes import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1488655105795_0014,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Preparation
Extract the Airports and Departure Delays information from WASBs

In [3]:
# Set File Paths
tripdelaysFilePath = "wasb://data@doctorwhostore.blob.core.windows.net/departuredelays.csv"
airportsnaFilePath = "wasb://data@doctorwhostore.blob.core.windows.net/airport-codes-na.txt"

# Obtain airports dataset
airportsna = spark.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', delimiter='\t').load(airportsnaFilePath)
airportsna.createOrReplaceTempView("airports_na")

# Obtain departure Delays data
departureDelays = spark.read.format("com.databricks.spark.csv").options(header='true').load(tripdelaysFilePath)
departureDelays.createOrReplaceTempView("departureDelays")
departureDelays.cache()

# Available IATA codes from the departuredelays sample dataset
tripIATA = spark.sql("select distinct iata from (select distinct origin as iata from departureDelays union all select distinct destination as iata from departureDelays) a")
tripIATA.createOrReplaceTempView("tripIATA")

# Only include airports with atleast one trip from the departureDelays dataset
airports = spark.sql("select f.IATA, f.City, f.State, f.Country from airports_na f join tripIATA t on t.IATA = f.IATA")
airports.createOrReplaceTempView("airports")
airports.cache()

DataFrame[IATA: string, City: string, State: string, Country: string]

In [4]:
# Build `departureDelays_geo` DataFrame
#  Obtain key attributes such as Date of flight, delays, distance, and airport information (Origin, Destination)  
departureDelays_geo = spark.sql("select cast(f.date as int) as tripid, cast(concat(concat(concat(concat(concat(concat('2014-', concat(concat(substr(cast(f.date as string), 1, 2), '-')), substr(cast(f.date as string), 3, 2)), ' '), substr(cast(f.date as string), 5, 2)), ':'), substr(cast(f.date as string), 7, 2)), ':00') as timestamp) as `localdate`, cast(f.delay as int), cast(f.distance as int), f.origin as src, f.destination as dst, o.city as city_src, d.city as city_dst, o.state as state_src, d.state as state_dst from departuredelays f join airports o on o.iata = f.origin join airports d on d.iata = f.destination") 

# Create Temporary View and cache
departureDelays_geo.createOrReplaceTempView("departureDelays_geo")
departureDelays_geo.cache()

# Count
departureDelays_geo.count()

1361141

In [30]:
%%sql
select * from departureDelays_geo limit 10

## Building the Graph
Now that we've imported our data, we're going to need to build our graph. To do so we're going to do two things: we are going to build the structure of the vertices (or nodes) and we're going to build the structure of the edges. What's awesome about GraphFrames is that this process is incredibly simple. 
* Rename IATA airport code to **id** in the Vertices Table
* Start and End airports to **src** and **dst** for the Edges Table (flights)

These are required naming conventions for vertices and edges in GraphFrames as of the time of this writing (Feb. 2016).

**WARNING:** If the graphframes package, required in the cell below.

In [6]:
# Note, ensure you have already installed the GraphFrames spack-package
from pyspark.sql.functions import *
from graphframes import *

# Create Vertices (airports) and Edges (flights)
tripVertices = airports.withColumnRenamed("IATA", "id").distinct()
tripEdges = departureDelays_geo.select("tripid", "delay", "src", "dst", "city_dst", "state_dst")

# Cache Vertices and Edges
tripEdges.cache()
tripVertices.cache()

DataFrame[id: string, City: string, State: string, Country: string]

In [7]:
# Vertices
#   The vertices of our graph are the airports
tripVertices.show()

+---+----------------+-----+-------+
| id|            City|State|Country|
+---+----------------+-----+-------+
|FAT|          Fresno|   CA|    USA|
|PHX|         Phoenix|   AZ|    USA|
|CMH|        Columbus|   OH|    USA|
|PAH|         Paducah|   KY|    USA|
|COS|Colorado Springs|   CO|    USA|
|RNO|            Reno|   NV|    USA|
|MYR|    Myrtle Beach|   SC|    USA|
|PSC|           Pasco|   WA|    USA|
|CAE|        Columbia|   SC|    USA|
|BPT|        Beaumont|   TX|    USA|
|SRQ|        Sarasota|   FL|    USA|
|VLD|        Valdosta|   GA|    USA|
|LAX|     Los Angeles|   CA|    USA|
|DAY|          Dayton|   OH|    USA|
|AVP|    Wilkes-Barre|   PA|    USA|
|MFR|         Medford|   OR|    USA|
|JFK|        New York|   NY|    USA|
|BNA|       Nashville|   TN|    USA|
|CLT|       Charlotte|   NC|    USA|
|LAS|       Las Vegas|   NV|    USA|
+---+----------------+-----+-------+
only showing top 20 rows

In [8]:
# Edges
#  The edges of our graph are the flights between airports
tripEdges.show()

+-------+-----+---+---+-------------------+---------+
| tripid|delay|src|dst|           city_dst|state_dst|
+-------+-----+---+---+-------------------+---------+
|1011111|   -5|MSP|INL|International Falls|       MN|
|1021111|    7|MSP|INL|International Falls|       MN|
|1031111|    0|MSP|INL|International Falls|       MN|
|1041925|    0|MSP|INL|International Falls|       MN|
|1061115|   33|MSP|INL|International Falls|       MN|
|1071115|   23|MSP|INL|International Falls|       MN|
|1081115|   -9|MSP|INL|International Falls|       MN|
|1091115|   11|MSP|INL|International Falls|       MN|
|1101115|   -3|MSP|INL|International Falls|       MN|
|1112015|   -7|MSP|INL|International Falls|       MN|
|1121925|   -5|MSP|INL|International Falls|       MN|
|1131115|   -3|MSP|INL|International Falls|       MN|
|1141115|   -6|MSP|INL|International Falls|       MN|
|1151115|   -7|MSP|INL|International Falls|       MN|
|1161115|   -3|MSP|INL|International Falls|       MN|
|1171115|    4|MSP|INL|Inter

In [9]:
# Build `tripGraph` GraphFrame
#  This GraphFrame builds up on the vertices and edges based on our trips (flights)
tripGraph = GraphFrame(tripVertices, tripEdges)
print tripGraph

# Build `tripGraphPrime` GraphFrame
#   This graphframe contains a smaller subset of data to make it easier to display motifs and subgraphs (below)
tripEdgesPrime = departureDelays_geo.select("tripid", "delay", "src", "dst")
tripGraphPrime = GraphFrame(tripVertices, tripEdgesPrime)

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 4 more fields])

## Simple Queries
Let's start with a set of simple graph queries to understand flight performance and departure delays

#### Determine the number of airports and trips

In [10]:
print "Airports: %d" % tripGraph.vertices.count()
print "Trips: %d" % tripGraph.edges.count()


Airports: 279
Trips: 1361141

#### Determining the longest delay in this dataset

In [11]:
tripGraph.edges.groupBy().max("delay").show()

+----------+
|max(delay)|
+----------+
|      1642|
+----------+

In [12]:
# Finding the longest Delay
tripGraph.edges.groupBy().max("delay").show()

+----------+
|max(delay)|
+----------+
|      1642|
+----------+

#### Determining the number of delayed vs. on-time / early flights

In [13]:
# Determining number of on-time / early flights vs. delayed flights
print "On-time / Early Flights: %d" % tripGraph.edges.filter("delay <= 0").count()
print "Delayed Flights: %d" % tripGraph.edges.filter("delay > 0").count()

On-time / Early Flights: 780469
Delayed Flights: 580672

#### What flights departing SEA are most likely to have significant delays
Note, delay can be <= 0 meaning the flight left on time or early

In [34]:
flightDelays = tripGraph.edges\
  .filter("src = 'SEA' and delay > 0")\
  .groupBy("src", "dst")\
  .avg("delay")\
  .sort(desc("avg(delay)"))
flightDelays.createOrReplaceTempView("flightDelays")

In [37]:
%%sql
select * from flightDelays order by `avg(delay)` desc limit 10

#### What destinations tend to have delays

In [15]:
# After displaying tripDelays, use Plot Options to set `state_dst` as a Key.
tripGraph.edges.filter("delay > 0").show()

+-------+-----+---+---+-------------------+---------+
| tripid|delay|src|dst|           city_dst|state_dst|
+-------+-----+---+---+-------------------+---------+
|1021111|    7|MSP|INL|International Falls|       MN|
|1061115|   33|MSP|INL|International Falls|       MN|
|1071115|   23|MSP|INL|International Falls|       MN|
|1091115|   11|MSP|INL|International Falls|       MN|
|1171115|    4|MSP|INL|International Falls|       MN|
|3031115|   17|MSP|INL|International Falls|       MN|
|3171115|   25|MSP|INL|International Falls|       MN|
|3181115|    2|MSP|INL|International Falls|       MN|
|3271115|    9|MSP|INL|International Falls|       MN|
|2091925|    1|MSP|INL|International Falls|       MN|
|2152015|   16|MSP|INL|International Falls|       MN|
|2161925|  169|MSP|INL|International Falls|       MN|
|2171115|   27|MSP|INL|International Falls|       MN|
|2181115|   96|MSP|INL|International Falls|       MN|
|2281115|    5|MSP|INL|International Falls|       MN|
|3021218|    2|IAH|MSY|     

#### What destinations tend to have significant delays departing from SEA

In [16]:
# States with the longest cumulative delays (with individual delays > 100 minutes) (origin: Seattle)
tripGraph.edges.filter("src = 'SEA' and delay > 100").show()

+-------+-----+---+---+-------------+---------+
| tripid|delay|src|dst|     city_dst|state_dst|
+-------+-----+---+---+-------------+---------+
|3201938|  108|SEA|BUR|      Burbank|       CA|
|3201655|  107|SEA|SNA|Orange County|       CA|
|2282055|  150|SEA|OAK|      Oakland|       CA|
|3061600|  130|SEA|OAK|      Oakland|       CA|
|1011950|  123|SEA|OAK|      Oakland|       CA|
|1021950|  194|SEA|OAK|      Oakland|       CA|
|1021615|  317|SEA|OAK|      Oakland|       CA|
|1021755|  385|SEA|OAK|      Oakland|       CA|
|1031950|  283|SEA|OAK|      Oakland|       CA|
|1031615|  364|SEA|OAK|      Oakland|       CA|
|1031325|  130|SEA|OAK|      Oakland|       CA|
|1061755|  107|SEA|OAK|      Oakland|       CA|
|1081330|  118|SEA|OAK|      Oakland|       CA|
|3170815|  199|SEA|DCA|Washington DC|     null|
|2151845|  128|SEA|KTN|    Ketchikan|       AK|
|2281845|  104|SEA|KTN|    Ketchikan|       AK|
|3130720|  117|SEA|KTN|    Ketchikan|       AK|
|2141157|  127|SEA|IAH|      Houston|   

## Vertex Degrees
* `inDegrees`: Incoming connections to the airport
* `outDegrees`: Outgoing connections from the airport 
* `degrees`: Total connections to and from the airport

Reviewing the various properties of the property graph to understand the incoming and outgoing connections between airports.

In [17]:
# Degrees
#  The number of degrees - the number of incoming and outgoing connections - for various airports within this sample dataset
tripGraph.degrees.sort(desc("degree")).limit(20).show()

+---+------+
| id|degree|
+---+------+
|ATL|179774|
|DFW|133966|
|ORD|125405|
|LAX|106853|
|DEN|103699|
|IAH| 85685|
|PHX| 79672|
|SFO| 77635|
|LAS| 66101|
|CLT| 56103|
|EWR| 54407|
|MCO| 54300|
|LGA| 50927|
|SLC| 50780|
|BOS| 49936|
|DTW| 46705|
|MSP| 46235|
|SEA| 45816|
|JFK| 43661|
|BWI| 42526|
+---+------+

In [18]:
# inDegrees
#  The number of degrees - the number of incoming connections - for various airports within this sample dataset
tripGraph.inDegrees.sort(desc("inDegree")).limit(20).show()

+---+--------+
| id|inDegree|
+---+--------+
|ATL|   89633|
|DFW|   65767|
|ORD|   61654|
|LAX|   53184|
|DEN|   50738|
|IAH|   42512|
|PHX|   39619|
|SFO|   38641|
|LAS|   32994|
|CLT|   28044|
|EWR|   27201|
|MCO|   27071|
|LGA|   25469|
|SLC|   25169|
|BOS|   24973|
|DTW|   23297|
|SEA|   22906|
|MSP|   22372|
|JFK|   21832|
|BWI|   21262|
+---+--------+

In [19]:
# outDegrees
#  The number of degrees - the number of outgoing connections - for various airports within this sample dataset
tripGraph.outDegrees.sort(desc("outDegree")).limit(20).show()

+---+---------+
| id|outDegree|
+---+---------+
|ATL|    90141|
|DFW|    68199|
|ORD|    63751|
|LAX|    53669|
|DEN|    52961|
|IAH|    43173|
|PHX|    40053|
|SFO|    38994|
|LAS|    33107|
|CLT|    28059|
|MCO|    27229|
|EWR|    27206|
|SLC|    25611|
|LGA|    25458|
|BOS|    24963|
|MSP|    23863|
|DTW|    23408|
|SEA|    22910|
|JFK|    21829|
|BWI|    21264|
+---+---------+

## City / Flight Relationships through Motif Finding
To more easily understand the complex relationship of city airports and their flights with each other, we can use motifs to find patterns of airports (i.e. vertices) connected by flights (i.e. edges). The result is a DataFrame in which the column names are given by the motif keys.

#### What delays might we blame on SFO

In [20]:
# Using tripGraphPrime to more easily display 
#   - The associated edge (ab, bc) relationships 
#   - With the different the city / airports (a, b, c) where SFO is the connecting city (b)
#   - Ensuring that flight ab (i.e. the flight to SFO) occured before flight bc (i.e. flight leaving SFO)
#   - Note, TripID was generated based on time in the format of MMDDHHMM converted to int
#       - Therefore bc.tripid < ab.tripid + 10000 means the second flight (bc) occured within approx a day of the first flight (ab)
# Note: In reality, we would need to be more careful to link trips ab and bc.
motifs = tripGraphPrime.find("(a)-[ab]->(b); (b)-[bc]->(c)")\
  .filter("(b.id = 'SFO') and (ab.delay > 500 or bc.delay > 500) and bc.tripid > ab.tripid and bc.tripid < ab.tripid + 10000")
motifs.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  ab|                   b|                  bc|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[DCA,Washington D...|[2030820,25,DCA,SFO]|[SFO,San Francisc...|[2030906,516,SFO,...|[PHL,Philadelphia...|
|[DCA,Washington D...|[2090823,-4,DCA,SFO]|[SFO,San Francisc...|[2092110,740,SFO,...|  [MIA,Miami,FL,USA]|
|[DCA,Washington D...|[2090823,-4,DCA,SFO]|[SFO,San Francisc...|[2092230,636,SFO,...|[JFK,New York,NY,...|
|[DCA,Washington D...| [2130805,0,DCA,SFO]|[SFO,San Francisc...|[2131420,504,SFO,...|[SAN,San Diego,CA...|
|[DCA,Washington D...|[2190805,-1,DCA,SFO]|[SFO,San Francisc...|[2190925,1638,SFO...|[ORD,Chicago,IL,USA]|
|[DCA,Washington D...|[2021700,110,DCA,...|[SFO,San Francisc...|[2030906,516,SFO,...|[PHL,Philadelphia...|
|[DCA,Washington D...|[2091700,95,DCA

## Determining Airport Ranking using PageRank
There are a large number of flights and connections through these various airports included in this Departure Delay Dataset.  Using the `pageRank` algorithm, Spark iteratively traverses the graph and determines a rough estimate of how important the airport is.

In [21]:
# Determining Airport ranking of importance using `pageRank`
ranks = tripGraph.pageRank(resetProbability=0.15, maxIter=5)
ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(10).show()

+---+--------------+-----+-------+------------------+
| id|          City|State|Country|          pagerank|
+---+--------------+-----+-------+------------------+
|ATL|       Atlanta|   GA|    USA|10.102340247484168|
|DFW|        Dallas|   TX|    USA|7.2520672596498175|
|ORD|       Chicago|   IL|    USA| 7.165214941660683|
|DEN|        Denver|   CO|    USA|5.0412555734844755|
|LAX|   Los Angeles|   CA|    USA| 4.178333397887623|
|IAH|       Houston|   TX|    USA|  4.00816934317367|
|SFO| San Francisco|   CA|    USA|3.5185952036532795|
|SLC|Salt Lake City|   UT|    USA|3.3564822581623024|
|PHX|       Phoenix|   AZ|    USA|3.0896771274957437|
|LAS|     Las Vegas|   NV|    USA| 2.437744837094788|
+---+--------------+-----+-------+------------------+

## Most popular flights (single city hops)
Using the `tripGraph`, we can quickly determine what are the most popular single city hop flights

In [38]:
# Determine the most popular flights (single city hops)
import pyspark.sql.functions as func
topTrips = tripGraph \
  .edges \
  .groupBy("src", "dst") \
  .agg(func.count("delay").alias("trips")) 
topTrips.createOrReplaceTempView("topTrips")

In [40]:
%%sql
select * from topTrips order by trips desc limit 10

In [23]:
# Show the top 20 most popular flights (single city hops)
topTrips.orderBy(topTrips.trips.desc()).limit(10).show()

+---+---+-----+
|src|dst|trips|
+---+---+-----+
|SFO|LAX| 3232|
|LAX|SFO| 3198|
|LAS|LAX| 3016|
|LAX|LAS| 2964|
|JFK|LAX| 2720|
|LAX|JFK| 2719|
|ATL|LGA| 2501|
|LGA|ATL| 2500|
|LAX|PHX| 2394|
|PHX|LAX| 2387|
+---+---+-----+

## Top Transfer Cities
Many airports are used as transfer points instead of the final Destination.  An easy way to calculate this is by calculating the ratio of inDegree (the number of flights to the airport) / outDegree (the number of flights leaving the airport).  Values close to 1 may indicate many transfers, whereas values < 1 indicate many outgoing flights and > 1 indicate many incoming flights.  Note, this is a simple calculation that does not take into account of timing or scheduling of flights, just the overall aggregate number within the dataset.

In [24]:
# Calculate the inDeg (flights into the airport) and outDeg (flights leaving the airport)
inDeg = tripGraph.inDegrees
outDeg = tripGraph.outDegrees

# Calculate the degreeRatio (inDeg/outDeg)
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id) \
  .drop(outDeg.id) \
  .selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio") \
  .cache()

# Join back to the `airports` DataFrame (instead of registering temp table as above)
nonTransferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA) \
  .selectExpr("id", "city", "degreeRatio") \
  .filter("degreeRatio < .9 or degreeRatio > 1.1")

# List out the city airports which have abnormal degree ratios.
nonTransferAirports.show()

+---+-----------+-------------------+
| id|       city|        degreeRatio|
+---+-----------+-------------------+
|GFK|Grand Forks| 1.3333333333333333|
|FAI|  Fairbanks| 1.1232686980609419|
|OME|       Nome| 0.5084745762711864|
|BRW|     Barrow|0.28651685393258425|
+---+-----------+-------------------+

In [25]:
# Join back to the `airports` DataFrame (instead of registering temp table as above)
transferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA) \
  .selectExpr("id", "city", "degreeRatio") \
  .filter("degreeRatio between 0.9 and 1.1")
  
# List out the top 10 transfer city airports
transferAirports.orderBy("degreeRatio").limit(10).show()

+---+--------------+------------------+
| id|          city|       degreeRatio|
+---+--------------+------------------+
|MSP|   Minneapolis|0.9375183338222353|
|DEN|        Denver| 0.958025717037065|
|DFW|        Dallas| 0.964339653074092|
|ORD|       Chicago|0.9671063983310065|
|SLC|Salt Lake City|0.9827417906368358|
|IAH|       Houston|0.9846895050147083|
|PHX|       Phoenix|0.9891643572266746|
|OGG| Kahului, Maui|0.9898718478710211|
|HNL|Honolulu, Oahu| 0.990535889872173|
|SFO| San Francisco|0.9909473252295224|
+---+--------------+------------------+

## Breadth First Search 
Breadth-first search (BFS) is designed to traverse the graph to quickly find the desired vertices (i.e. airports) and edges (i.e flights).  Let's try to find the shortest number of connections between cities based on the dataset.  Note, these examples do not take into account of time or distance, just hops between cities.

In [26]:
# Example 1: Direct Seattle to San Francisco 
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SEA'",
  toExpr = "id = 'SFO'",
  maxPathLength = 1)
filteredPaths.show()

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[SEA,Seattle,WA,USA]|[2010720,-2,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2012125,-3,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2011845,-4,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2011220,-5,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2010955,-4,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2011405,-2,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2020720,21,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2022125,7,SEA,SF...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2021845,1,SEA,SF...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2020610,-8,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2021220,67,SEA,S...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[2021100,0,SEA,SF...|[SFO,San Francisc...|
|[SEA,Seattle,WA,USA]|[20

As you can see, there are a number of direct flights between Seattle and San Francisco.

In [27]:
# Example 2: Direct San Francisco and Buffalo
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 1)
filteredPaths.show()

+---+----+-----+-------+
| id|City|State|Country|
+---+----+-----+-------+
+---+----+-----+-------+

But there are no direct flights between San Francisco and Buffalo.

In [28]:
# Example 2a: Flying from San Francisco to Buffalo
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 2)
filteredPaths.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+
|                from|                  e0|                 v1|                  e1|                  to|
+--------------------+--------------------+-------------------+--------------------+--------------------+
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BOS,Boston,MA,USA]|[3010630,-4,BOS,B...|[BUF,Buffalo,NY,USA]|
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BOS,Boston,MA,USA]|[3011445,8,BOS,BU...|[BUF,Buffalo,NY,USA]|
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BOS,Boston,MA,USA]|[3020630,-5,BOS,B...|[BUF,Buffalo,NY,USA]|
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BOS,Boston,MA,USA]|[3021049,-7,BOS,B...|[BUF,Buffalo,NY,USA]|
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BOS,Boston,MA,USA]|[3021445,-3,BOS,B...|[BUF,Buffalo,NY,USA]|
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BOS,Boston,MA,USA]|[3030630,0,BOS,BU...|[BUF,Buffalo,NY,USA]|
|[SFO,San Francisc...|[2010700,5,SFO,BO...|[BO

But there are flights from San Francisco to Buffalo with Minneapolis as the transfer point.  But there are flights from San Francisco to Buffalo with Minneapolis as the transfer point. But what are the most popular layovers between SFO and BUF?

In [29]:
filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).limit(10).show()

+---+---------------+-------+
| id|           City|  count|
+---+---------------+-------+
|JFK|       New York|1233728|
|ORD|        Chicago|1088283|
|ATL|        Atlanta| 285383|
|LAS|      Las Vegas| 275091|
|BOS|         Boston| 238576|
|CLT|      Charlotte| 143444|
|PHX|        Phoenix| 104580|
|FLL|Fort Lauderdale|  96317|
|EWR|         Newark|  95370|
|MCO|        Orlando|  88615|
+---+---------------+-------+